In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

import json
import numpy as np

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
class TextClassifier(nn.Module):
    def __init__(
        self,
        num_classes,
        vocab_size,
        embedding_dim,
        embedding_tensors,
        lstm_hidden_size,
        conv_in_channels,
        conv_out_channels,
        fc_in_features,
        fc_out_features
    ):
        super(TextClassifier, self).__init__()

        # Embedding layer
        if embedding_tensors is not None:
          self.embedding = nn.Embedding.from_pretrained(embedding_tensors, freeze=False).to(device)
        else:
          self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)

        # Dropout
        self.dropout = nn.Dropout(0.2)

        # LSTM
        self.LSTM = nn.LSTM(input_size=embedding_dim, hidden_size=lstm_hidden_size, batch_first=True)

        # CNN
        self.Conv1 = nn.Conv1d(in_channels=conv_in_channels, out_channels=conv_out_channels, kernel_size=3)
        self.Conv2 = nn.Conv1d(in_channels=conv_in_channels, out_channels=conv_out_channels, kernel_size=5)
        self.Conv3 = nn.Conv1d(in_channels=conv_in_channels, out_channels=conv_out_channels, kernel_size=7)

        # Fully connected layer
        self.fc = nn.Linear(in_features=fc_in_features, out_features=fc_out_features)

        # Output layer
        self.output = nn.Linear(in_features=fc_out_features, out_features=num_classes)

    def forward(self, x):
        embeds = self.embedding(x) # (batch_size, sequence_length, embedding_dim)
        lstm_out, _ = self.LSTM(embeds) # (batch_size, sequence_length, 128)
        lstm_out = lstm_out[:, -1, :] # Select the last time step in each batch

        embeds = embeds.permute(0, 2, 1) # (batch_size, embedding_dim, sequence_length)
        conv1_out = F.relu(self.Conv1(embeds))
        conv1_out = torch.max(conv1_out, dim=2)[0] # (batch_size, conv_out_channels)

        conv2_out = F.relu(self.Conv2(embeds))
        conv2_out = torch.max(conv2_out, dim=2)[0]

        conv3_out = F.relu(self.Conv3(embeds))
        conv3_out = torch.max(conv3_out, dim=2)[0]

        conv_concat = torch.cat((conv1_out, conv2_out, conv3_out), dim=1) # (batch_size, 3 * conv_out_channels)

        lstm_cnn_concat = torch.cat((lstm_out, conv_concat), dim=1) # (batch_size, 3 * conv_out_channels + 128)
        fc_out = self.dropout(F.relu(self.fc(lstm_cnn_concat)))
        output = self.output(fc_out)
        return output

In [3]:
def encode_sentence(sentence: str, word_to_index: dict, max_tokens: int) -> list:
    fixed_encoded = np.zeros(max_tokens, dtype=int)
    encoded = np.array([word_to_index.get(word, len(word_to_index)) for word in sentence.split(" ")])

    # Ensure the encoded sentence reaches the maximum specified length.
    length = min(max_tokens, len(encoded))
    fixed_encoded[:length] = encoded[:length]
    return list(fixed_encoded)

In [4]:
lstm_cnn_model = torch.load("../models/lstm_cnn_model.pth", map_location="cpu", weights_only=False)["model"]

with open("../models/word_to_index.json", "r") as f:
    word_to_index = json.load(f)
    word_to_index = {k: int(v) for k, v in word_to_index.items()}

In [5]:
label_map = {
    0: "Negative",
    1: "Neutral",
    2: "Positive"
}

sentence = "Thầy dạy nhanh, khó tiếp thu."
true_label = 0

encodings = torch.tensor(encode_sentence(sentence, word_to_index, max_tokens=200)).unsqueeze(0)
logits = lstm_cnn_model(encodings)
predicted_label = torch.argmax(torch.softmax(logits, dim=1), dim=1).item()

print("Sentence: ", sentence)
print("True Label: ", label_map[true_label])
print("Predicted Label: ", label_map[predicted_label])

Sentence:  Thầy dạy nhanh, khó tiếp thu.
True Label:  Negative
Predicted Label:  Negative
